In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import joblib
%store -r X_features
%store -r X_features_poly
%store -r y_labels
from sklearn.metrics import mean_absolute_percentage_error


In [2]:
from sklearn.ensemble import VotingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor

from sklearn.linear_model import Ridge
from sklearn.svm import LinearSVR



In [3]:
svm_rbf_reg = SVR(kernel="rbf",C=1,epsilon=0.001)
forest_reg = RandomForestRegressor(n_estimators=150,max_samples=5000)

poly_svm_rbf_reg = SVR(kernel="rbf",C=1,epsilon=0.001)
poly_forest_reg = RandomForestRegressor(n_estimators=100)

voting_reg = VotingRegressor(
    estimators=[('fr', forest_reg), ('svm', svm_rbf_reg)])

poly_voting_reg = VotingRegressor(
    estimators=[('pfr', poly_svm_rbf_reg), ('psvm', poly_forest_reg)])

In [4]:
voting_reg.fit(X_features[0],y_labels[0])
poly_voting_reg.fit(X_features_poly[0],y_labels[0])

VotingRegressor(estimators=[('pfr', SVR(C=1, epsilon=0.001)),
                            ('psvm', RandomForestRegressor())])

In [5]:
from sklearn.metrics import mean_absolute_percentage_error

visc_predictions = np.exp(np.power(poly_voting_reg.predict(X_features_poly[1]),2))-0.0012
poly_voting_mse = mean_absolute_percentage_error(np.exp(np.power(y_labels[1],2))-0.0012, visc_predictions)
poly_voting_rmse = np.sqrt(poly_voting_mse)
poly_voting_mse

0.033865096618308825

In [6]:
joblib.dump(poly_voting_reg, "poly_voting_reg.pkl")

['poly_voting_reg.pkl']

In [7]:

visc_predictions = np.exp(np.power(voting_reg.predict(X_features[1]),2))-0.0012
voting_mse = mean_absolute_percentage_error(np.exp(np.power(y_labels[1],2))-0.0012, visc_predictions)
voting_rmse = np.sqrt(voting_mse)
voting_mse

0.033823665569633274

In [8]:
joblib.dump(voting_reg, "voting_reg.pkl")

['voting_reg.pkl']

### Εδώ πραγματοποιούμε τη μέθοδο bagging για ένα δέντρο η οποία υπό συγκεκριμένες συνθήκες ταυτίζεται με τον random forest regressor.

In [9]:
from sklearn.ensemble import BaggingRegressor

bag_reg = BaggingRegressor(
    DecisionTreeRegressor( ), n_estimators=200,
    max_samples=5000, bootstrap=True)
bag_reg.fit(X_features_poly[0],y_labels[0])

BaggingRegressor(base_estimator=DecisionTreeRegressor(), max_samples=5000,
                 n_estimators=200)

In [10]:
joblib.dump(bag_reg, "bag_reg.pkl")

['bag_reg.pkl']

In [11]:
from sklearn.ensemble import AdaBoostRegressor

ada_reg = AdaBoostRegressor(
    DecisionTreeRegressor(), n_estimators=200,
    learning_rate=0.5)
ada_reg.fit(X_features_poly[0],y_labels[0])

AdaBoostRegressor(base_estimator=DecisionTreeRegressor(), learning_rate=0.5,
                  n_estimators=200)

In [12]:
joblib.dump(ada_reg, "ada_reg.pkl")

['ada_reg.pkl']

### Εδώ θα χρησιμοποιήσουμε τη μέθοδο του Boosting.

In [13]:
import xgboost
xgb_reg = xgboost.XGBRegressor(max_depth = 16,learning_rate=0.1)
xgb_reg.fit(X_features_poly[0],y_labels[0],
eval_set=[(X_features_poly[1],y_labels[1])])



[0]	validation_0-rmse:0.21634
[1]	validation_0-rmse:0.19673
[2]	validation_0-rmse:0.17922
[3]	validation_0-rmse:0.16379
[4]	validation_0-rmse:0.15006
[5]	validation_0-rmse:0.13800
[6]	validation_0-rmse:0.12762
[7]	validation_0-rmse:0.11834
[8]	validation_0-rmse:0.11035
[9]	validation_0-rmse:0.10356
[10]	validation_0-rmse:0.09777
[11]	validation_0-rmse:0.09285
[12]	validation_0-rmse:0.08866
[13]	validation_0-rmse:0.08521
[14]	validation_0-rmse:0.08239
[15]	validation_0-rmse:0.08003
[16]	validation_0-rmse:0.07806
[17]	validation_0-rmse:0.07650
[18]	validation_0-rmse:0.07526
[19]	validation_0-rmse:0.07428
[20]	validation_0-rmse:0.07348
[21]	validation_0-rmse:0.07291
[22]	validation_0-rmse:0.07246
[23]	validation_0-rmse:0.07216
[24]	validation_0-rmse:0.07195
[25]	validation_0-rmse:0.07179
[26]	validation_0-rmse:0.07167
[27]	validation_0-rmse:0.07158
[28]	validation_0-rmse:0.07155
[29]	validation_0-rmse:0.07153
[30]	validation_0-rmse:0.07157
[31]	validation_0-rmse:0.07156
[32]	validation_0-

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.1, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=16, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, ...)

In [14]:
joblib.dump(xgb_reg, "xgb_reg.pkl")

['xgb_reg.pkl']

In [15]:
import lightgbm as lgb
t_data = lgb.Dataset(X_features[0],label=y_labels[0])
v_data = lgb.Dataset(X_features[1],label=y_labels[1])

params = {
    'task': 'train', 
    'boosting': 'gbdt',
    'objective': 'regression',
    'num_leaves': 10,
    'learning_rage': 0.5,
    'metric': {'l2','l1'},
    'verbose': -1
}
light = lgb.train(params,
                 train_set=t_data,
                 valid_sets=v_data,)
  

[LightGBM] [Warning] Unknown parameter: learning_rage
[1]	valid_0's l2: 0.0233153	valid_0's l1: 0.120725
[2]	valid_0's l2: 0.0201618	valid_0's l1: 0.111689
[3]	valid_0's l2: 0.0175451	valid_0's l1: 0.103697
[4]	valid_0's l2: 0.0154375	valid_0's l1: 0.0965875
[5]	valid_0's l2: 0.0137045	valid_0's l1: 0.0902615
[6]	valid_0's l2: 0.0122719	valid_0's l1: 0.0847464
[7]	valid_0's l2: 0.0110835	valid_0's l1: 0.0798596
[8]	valid_0's l2: 0.0101213	valid_0's l1: 0.0755696
[9]	valid_0's l2: 0.00933357	valid_0's l1: 0.0718796
[10]	valid_0's l2: 0.00867986	valid_0's l1: 0.0686684
[11]	valid_0's l2: 0.00814227	valid_0's l1: 0.0658878
[12]	valid_0's l2: 0.00769196	valid_0's l1: 0.063488
[13]	valid_0's l2: 0.00731804	valid_0's l1: 0.0614264
[14]	valid_0's l2: 0.00702876	valid_0's l1: 0.0597041
[15]	valid_0's l2: 0.00678671	valid_0's l1: 0.0582416
[16]	valid_0's l2: 0.00658616	valid_0's l1: 0.0569282
[17]	valid_0's l2: 0.0063887	valid_0's l1: 0.0557571
[18]	valid_0's l2: 0.00624581	valid_0's l1: 0.0548

In [16]:
joblib.dump(light, "light.pkl")

['light.pkl']

In [17]:
from sklearn.ensemble import StackingRegressor
estimators=[('fr', forest_reg), ('svm', svm_rbf_reg)]
reg = StackingRegressor(
     estimators=estimators,
     final_estimator=RandomForestRegressor(n_estimators=100))

In [18]:
reg.fit(X_features[0], y_labels[0])

StackingRegressor(estimators=[('fr',
                               RandomForestRegressor(max_samples=5000,
                                                     n_estimators=150)),
                              ('svm', SVR(C=1, epsilon=0.001))],
                  final_estimator=RandomForestRegressor())

In [19]:
joblib.dump(reg, "stack.pkl")

['stack.pkl']